<a href="https://colab.research.google.com/github/Sravaniparepalli/FMML-LABS/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [16]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [17]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [18]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [19]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [20]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [21]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [22]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [23]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [24]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [25]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [26]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [27]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [28]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [29]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?

Yes, averaging validation accuracy across multiple splits typically gives **more consistent and reliable results** compared to using a single validation split. This approach is part of **k-fold cross-validation**, a widely used method for model evaluation and selection. Here's why it works and how it helps:

---

### **1. Why Averaging Across Splits Helps**
1. **Reduces Variance Due to Data Splits**:  
   - A single train-test split might lead to biased results if the split happens to include an unrepresentative subset of the data (e.g., imbalanced classes or outliers).  
   - Averaging across multiple splits ensures that the model's performance is evaluated on diverse subsets, reducing the likelihood of overfitting to specific data.

2. **Improved Generalization Estimation**:  
   - By training and validating on different folds, you mimic real-world scenarios where the model is tested on unseen data.
   - The average accuracy is a better approximation of how the model will perform in general use cases.

3. **Better Use of Data**:  
   - k-fold cross-validation allows every data point to be used for both training and validation. This is especially beneficial when working with small datasets.

4. **Mitigates Overfitting**:  
   - A model evaluated on multiple folds is less likely to show artificially inflated accuracy caused by overfitting to a specific split.

---

### **2. How Averaging Works in k-Fold Cross-Validation**
- In **k-fold cross-validation**:
  1. The dataset is split into **k equal parts** (folds).
  2. The model is trained on \( k-1 \) folds and validated on the remaining fold.
  3. This process repeats \( k \) times, with each fold serving as the validation set once.
  4. The **final validation accuracy** is the average of the accuracies across all \( k \) folds.

---

### **3. Benefits of Averaging**
- **Consistency Across Splits**:
  - Averaging smooths out variations in accuracy caused by random splits.
  - If one split is "easy" or "hard" (e.g., it has a disproportionately high or low accuracy due to specific data points), its effect is mitigated by averaging.

- **Statistical Robustness**:
  - The average validation accuracy provides a more stable estimate, while individual fold accuracies give insight into variability.

---

### **4. Caveats and Considerations**
1. **Bias-Variance Tradeoff**:  
   - While k-fold reduces variance in accuracy estimation, the choice of \( k \) affects bias. A large \( k \) (e.g., \( k = 10 \)) typically offers a better balance.

2. **Compute Cost**:  
   - k-fold cross-validation is computationally intensive since the model is trained \( k \) times. This can be a concern for large datasets or complex models.

3. **Data Leakage Risks**:  
   - Ensure proper data splitting, especially when using techniques like stratified k-fold (preserving class distribution). Improper splits can lead to overestimation of accuracy.

---

### **5. When to Use Averaging**
- **Small to Medium Datasets**: Cross-validation with averaging is especially helpful when data is limited. It helps ensure that the model's performance is not overly dependent on any single split.
- **Hyperparameter Tuning**: Use the average validation accuracy to select the best set of hyperparameters.
- **Comparing Models**: Averaged metrics offer a fairer basis for comparing multiple models or architectures.

---

### **6. Example: Averaging Validation Accuracy**
For a k-fold setup with \( k = 5 \), validation accuracies might look like this:  

| Fold  | Validation Accuracy (%) |
|-------|--------------------------|
| Fold 1| 82                       |
| Fold 2| 85                       |
| Fold 3| 83                       |
| Fold 4| 84                       |
| Fold 5| 81                       |

**Average Accuracy** = \( \frac{82 + 85 + 83 + 84 + 81}{5} = 83% \)

The average provides a more reliable performance metric than relying on just one split.

---

Would you like help implementing cross-validation in code (e.g., Python using `scikit-learn`)?
2. Does it give more accurate estimate of test accuracy?
Yes, averaging validation accuracy across multiple splits (as done in **k-fold cross-validation**) generally provides a **more accurate estimate of test accuracy** than a single train-test split. Here's why this method is more reliable for estimating a model's performance on unseen data:

---

### **1. Reducing the Effect of Random Splits**
- In a single train-test split, the training and test datasets are determined randomly. If this split is unbalanced (e.g., test data is harder, or key patterns are missing in the training data), the accuracy estimate might be misleading.
- Averaging across multiple splits ensures the model is evaluated on different subsets of the data, reducing the influence of randomness. This leads to a performance estimate closer to the true generalization performance.

---

### **2. Accounting for Variability in the Data**
- In practice, real-world data can be noisy or diverse, and a single split might not capture this variability.
- Cross-validation exposes the model to different parts of the data during both training and validation, providing a broader evaluation of its ability to generalize to unseen data.

---

### **3. Balancing Bias and Variance**
- **High variance** in a single train-test split can lead to over- or underestimation of test accuracy.
- Cross-validation averages out these variations, leading to a **lower-variance estimate**.
- However, if the number of folds (\( k \)) is too small (e.g., \( k=2 \)), the estimate might still be biased. Larger \( k \) (e.g., \( k=10 \)) reduces this bias while maintaining a good tradeoff with variance.

---

### **4. Comparison of Estimates**
#### **Single Train-Test Split**
- Test accuracy depends entirely on the specific split.
- Example: If the split happens to include simpler examples in the test set, it may overestimate the model's true accuracy.

#### **k-Fold Cross-Validation**
- Each fold provides a unique test set, and the final averaged accuracy is typically closer to the model's true performance on unseen data.
- Example: A 10-fold cross-validation provides 10 validation accuracies, each evaluated on 10% of the data.

---

### **5. Situations Where It Helps**
1. **Small Datasets**:
   - When data is limited, a single test set might not represent the full variability of the dataset.
   - Cross-validation ensures every data point is used for both training and testing, maximizing information usage.

2. **Hyperparameter Tuning**:
   - A model tuned using a single split might overfit to that split.
   - Cross-validation helps find parameters that generalize well across multiple subsets.

3. **Model Selection**:
   - Comparing models on a single split might favor one due to random chance.
   - Cross-validation averages out these effects, providing a fair comparison.

---

### **6. Does It Always Improve Accuracy Estimates?**
While cross-validation typically gives a more accurate estimate, it does not eliminate all sources of error:
1. **Overfitting in Cross-Validation**:
   - If the model is too complex for the data, it can overfit even with cross-validation.
   - Use regularization or simpler models if cross-validated performance is still too optimistic compared to test performance.

2. **Distributional Shifts**:
   - Cross-validation assumes that the training and test data come from the same distribution. If the real test data is very different, even cross-validation might not provide an accurate estimate.

---

### **7. Final Estimate of Test Accuracy**
- The **cross-validated accuracy** is often the closest approximation of the test accuracy if:
  - The test data distribution matches the training data.
  - Cross-validation is done properly (e.g., stratified splits for imbalanced classes).
- However, for the most reliable performance assessment, always reserve a **final, unseen test set** for evaluation after cross-validation. This ensures that the cross-validation process itself hasn’t influenced the test accuracy.

---

Would you like an example or code snippet to demonstrate this?


3. Yes, averaging validation accuracy across multiple splits (as done in **k-fold cross-validation**) generally provides a **more accurate estimate of test accuracy** than a single train-test split. Here's why this method is more reliable for estimating a model's performance on unseen data:

---




4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

If you have a **very small training or validation dataset**, increasing the number of iterations (e.g., epochs in deep learning or training cycles in general) is not a direct solution to the problem. While increasing iterations can help the model learn more effectively from the limited data, it introduces potential risks, such as **overfitting**, because the model might memorize the small dataset instead of generalizing well. Here’s a detailed analysis:

---

### **Key Factors to Consider**
1. **Small Training Dataset**:
   - If your training dataset is small, the model has limited information to learn patterns that generalize to new data.
   - Simply increasing the number of iterations can make the model overly fit the training data, leading to poor performance on unseen data.

2. **Small Validation Dataset**:
   - A small validation dataset means your estimate of model performance will have **high variance**, as the validation results depend on a small, potentially unrepresentative sample.
   - Cross-validation can help here (e.g., k-fold cross-validation) by making better use of limited data for validation.

---

### **Does Increasing Iterations Help?**
Increasing iterations might improve the model **to an extent**, but it’s not a complete solution for small datasets. Here’s why:

#### **When It Might Help**:
1. **Underfitting**: If the model hasn’t fully captured patterns in the data, increasing iterations can help it converge.
2. **Small, Well-Represented Data**: If your dataset, though small, is representative of the problem space, more iterations can allow the model to extract the maximum information.

#### **When It Fails**:
1. **Overfitting**: More iterations can lead to the model memorizing the training data instead of generalizing patterns. This results in a gap between training and validation accuracy.
2. **Noise Sensitivity**: With small datasets, the model might overfit to noise or irrelevant patterns in the data.

---

### **Strategies to Deal with Small Datasets**
Instead of relying solely on increasing iterations, you can apply the following techniques to better utilize small datasets:

#### **1. Data Augmentation**
   - **What It Is**: Generate new training examples by applying transformations to existing data (e.g., rotations, flips, noise addition).
   - **Why It Helps**: Augmentation increases the effective size of the dataset and introduces variability, reducing overfitting.
   - **Examples**:
     - For images: Rotate, flip, crop, adjust brightness.
     - For text: Synonym replacement, paraphrasing.
     - For time-series: Add noise, stretch/compress signals.

#### **2. Regularization**
   - **What It Is**: Add penalties to the model to prevent it from becoming too complex (e.g., L1/L2 regularization, dropout in neural networks).
   - **Why It Helps**: Regularization discourages overfitting to the small dataset.

#### **3. Transfer Learning**
   - **What It Is**: Use a pre-trained model on a related problem, and fine-tune it on your small dataset.
   - **Why It Helps**: Pre-trained models already capture useful features, reducing the need for large training datasets.

#### **4. Cross-Validation**
   - **What It Is**: Use k-fold cross-validation to maximize the use of available data for training and validation.
   - **Why It Helps**: This reduces variance in validation results and gives a better estimate of model performance.

#### **5. Early Stopping**
   - **What It Is**: Monitor the validation loss during training and stop training when it stops improving.
   - **Why It Helps**: Prevents overfitting by halting training once the model has extracted the most relevant patterns.

#### **6. Synthetic Data Generation**
   - **What It Is**: Create synthetic samples using techniques like SMOTE (Synthetic Minority Oversampling Technique) for tabular data or GANs (Generative Adversarial Networks) for images.
   - **Why It Helps**: Provides additional data for training, improving generalization.

#### **7. Use Simpler Models**
   - **What It Is**: Use models with fewer parameters or simpler architectures.
   - **Why It Helps**: Complex models require more data to avoid overfitting.

---

### **Increasing Iterations in Context**
Increasing iterations can be beneficial **only when combined with techniques like regularization or data augmentation**. Here's a scenario where it works:
- **With Regularization**: If you're applying L2 regularization, increasing iterations can allow the model to explore the data thoroughly without overfitting.
- **With Augmentation**: Increasing iterations gives the model time to see all augmented variations of the data.

---

### **Practical Example**
Consider training a neural network with a small dataset of 1,000 samples:
1. **Without Augmentation**:
   - Training for 100 epochs might cause overfitting after the 20th epoch.
   - Increasing iterations to 200 would worsen generalization.

2. **With Augmentation**:
   - Adding noise, rotations, and scaling increases the effective dataset size to 10,000 samples.
   - Training for 100-200 epochs allows the model to learn from a richer data distribution.

---

### **Conclusion**
Increasing iterations can help to some extent, but by itself, it is not a complete solution for small datasets. Instead:
- Use **data augmentation**, **regularization**, or **transfer learning** to address the root problem.
- Combine these with **early stopping** to prevent overfitting during longer training.

Would you like help with implementing any of these strategies in practice?

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.